In [231]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

In [233]:
#1.資料前處理
#讀xls檔
df = pd.read_excel("新竹_2021.xls")
df

,測站,日期,測項,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
0,--------------------,--------------------,--------------------,-------------------------------,-------------------------------,-------------------------------,-------------------------------,-------------------------------,-------------------------------,-------------------------------,...,-------------------------------,-------------------------------,-------------------------------,-------------------------------,-------------------------------,-------------------------------,-------------------------------,-------------------------------,-------------------------------,-------------------------------
1,新竹,2021-01-01 00:00:00,AMB_TEMP,11.1,11.2,11.4,11.5,11.6,11.7,11.9,...,16.6,16.3,15.6,14.8,14.4,14.5,14.7,14.7,14.6,14.4
2,新竹,2021-01-01 00:00:00,CH4,2.01,1.99,2,2.02,2.03,2.02,2.02,...,1.98,1.97,1.97,2,2.02,2.01,2.01,2,1.98,1.98
3,新竹,2021-01-01 00:00:00,CO,0.31,0.28,0.28,0.33,0.32,0.26,0.25,...,0.31,0.29,0.29,0.33,0.34,0.34,0.34,0.29,0.24,0.21
4,新竹,2021-01-01 00:00:00,NMHC,0.1,0.1,0.08,0.09,0.1,0.07,0.07,...,0.06,0.07,0.08,0.12,0.13,0.1,0.1,0.09,0.05,0.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6566,新竹,2021-12-31 00:00:00,THC,2.24,2.22,2.19,2.17,2.15,2.1,2.1,...,2.07,2.05,2.09,2.1,2.05,2.1,2.15,2.13,2.09,2.05
6567,新竹,2021-12-31 00:00:00,WD_HR,38,51,50,47,53,53,46,...,51,54,48,53,54,53,47,37,42,48
6568,新竹,2021-12-31 00:00:00,WIND_DIREC,37,59,37,50,62,42,41,...,66,45,40,59,57,55,41,36,53,39
6569,新竹,2021-12-31 00:00:00,WIND_SPEED,2.6,2.6,2.3,2.4,3.4,3.2,3.1,...,4.8,3.2,2.8,3.2,2.5,2.2,1.7,2.5,2.3,1.9


In [235]:
#將欄位重新命名
hours = df.columns[3:].tolist()
df.columns = ['Site', 'Date', 'Target'] + hours
df.columns

Index([  'Site',   'Date', 'Target',        0,        1,        2,        3,
              4,        5,        6,        7,        8,        9,       10,
             11,       12,       13,       14,       15,       16,       17,
             18,       19,       20,       21,       22,       23],
      dtype='object')

In [237]:
# 去掉Site以及分隔線
df = df.drop(['Site'], axis=1).drop(0)
df

,Date,Target,0,1,2,3,4,5,6,7,...,14,15,16,17,18,19,20,21,22,23
1,2021-01-01 00:00:00,AMB_TEMP,11.1,11.2,11.4,11.5,11.6,11.7,11.9,12.1,...,16.6,16.3,15.6,14.8,14.4,14.5,14.7,14.7,14.6,14.4
2,2021-01-01 00:00:00,CH4,2.01,1.99,2,2.02,2.03,2.02,2.02,2.01,...,1.98,1.97,1.97,2,2.02,2.01,2.01,2,1.98,1.98
3,2021-01-01 00:00:00,CO,0.31,0.28,0.28,0.33,0.32,0.26,0.25,0.27,...,0.31,0.29,0.29,0.33,0.34,0.34,0.34,0.29,0.24,0.21
4,2021-01-01 00:00:00,NMHC,0.1,0.1,0.08,0.09,0.1,0.07,0.07,0.08,...,0.06,0.07,0.08,0.12,0.13,0.1,0.1,0.09,0.05,0.06
5,2021-01-01 00:00:00,NO,1.5,1.4,1.4,1.5,1.4,1.3,1.4,1.9,...,3.5,2.6,2.3,2,1.8,1.8,1.8,1.7,1.5,1.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6566,2021-12-31 00:00:00,THC,2.24,2.22,2.19,2.17,2.15,2.1,2.1,2.11,...,2.07,2.05,2.09,2.1,2.05,2.1,2.15,2.13,2.09,2.05
6567,2021-12-31 00:00:00,WD_HR,38,51,50,47,53,53,46,49,...,51,54,48,53,54,53,47,37,42,48
6568,2021-12-31 00:00:00,WIND_DIREC,37,59,37,50,62,42,41,59,...,66,45,40,59,57,55,41,36,53,39
6569,2021-12-31 00:00:00,WIND_SPEED,2.6,2.6,2.3,2.4,3.4,3.2,3.1,3,...,4.8,3.2,2.8,3.2,2.5,2.2,1.7,2.5,2.3,1.9


In [239]:
#1.1 取出 10.11.12 月資料

df.Date = pd.to_datetime(df.Date)
df = df[df.Date >= '2021-10-01']
df = df.reset_index(drop=True)
df

,Date,Target,0,1,2,3,4,5,6,7,...,14,15,16,17,18,19,20,21,22,23
0,2021-10-01,AMB_TEMP,28.3,28.3,27.8,27.8,27.6,27.6,27.7,28.4,...,31.6,31.4,30.9,30.5,30.2,29.8,29.4,29.1,28.7,28.2
1,2021-10-01,CH4,2.04,2.02,2.12,2.18,2.19,2.24,2.21,2.17,...,1.96,1.97,2.01,2.06,2.07,2.05,2.04,2.03,2.08,2.08
2,2021-10-01,CO,0.34,0.3,0.3,0.29,0.3,0.33,0.44,0.62,...,0.25,0.27,0.32,0.43,0.45,0.45,0.43,0.42,0.43,0.39
3,2021-10-01,NMHC,0.17,0.13,0.12,0.14,0.17,0.16,0.18,0.23,...,0.04,0.06,0.05,0.17,0.24,0.22,0.16,0.14,0.16,0.14
4,2021-10-01,NO,0.9,0.2,0.5,0.4,0.2,0.6,2.2,3.6,...,0.5,0.5,0.5,0.3,0.3,0.3,0.3,0.3,0.4,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,2021-12-31,THC,2.24,2.22,2.19,2.17,2.15,2.1,2.1,2.11,...,2.07,2.05,2.09,2.1,2.05,2.1,2.15,2.13,2.09,2.05
1652,2021-12-31,WD_HR,38,51,50,47,53,53,46,49,...,51,54,48,53,54,53,47,37,42,48
1653,2021-12-31,WIND_DIREC,37,59,37,50,62,42,41,59,...,66,45,40,59,57,55,41,36,53,39
1654,2021-12-31,WIND_SPEED,2.6,2.6,2.3,2.4,3.4,3.2,3.1,3,...,4.8,3.2,2.8,3.2,2.5,2.2,1.7,2.5,2.3,1.9


In [241]:
# 去掉字串後多餘的空白
df['Target'] = df['Target'].apply(lambda x: x.strip())

# 依類別分組
sector = df.groupby('Target')
sector.size()

Target
AMB_TEMP      92
CH4           92
CO            92
NMHC          92
NO            92
NO2           92
NOx           92
O3            92
PM10          92
PM2.5         92
RAINFALL      92
RH            92
SO2           92
THC           92
WD_HR         92
WIND_DIREC    92
WIND_SPEED    92
WS_HR         92
dtype: int64

In [243]:
# df_all: 欄名為類別 欄位內容為二維轉一維數據

df_all = pd.DataFrame()
categorys =df['Target'].unique()

for category in categorys:
    temp = sector.get_group(category).loc[:, 0:]
    df_all[category] = temp.values.reshape(-1)

df_all

,AMB_TEMP,CH4,CO,NMHC,NO,NO2,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,THC,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR
0,28.3,2.04,0.34,0.17,0.9,18.8,19.8,16,28,28,0,71,1.8,2.21,62,33,0.8,0.7
1,28.3,2.02,0.3,0.13,0.2,11.9,12.2,21.5,24,22,0,66,1,2.15,121,183,1.2,0.6
2,27.8,2.12,0.3,0.12,0.5,15.1,15.6,16.9,29,26,0,76,1,2.24,164,160,1.1,0.6
3,27.8,2.18,0.29,0.14,0.4,12.8,13.2,16.4,32,24,0,79,1.3,2.32,156,151,0.6,0.4
4,27.6,2.19,0.3,0.17,0.2,14.9,15.1,12.6,31,28,0,81,1.5,2.36,110,90,0.9,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203,15.8,2.01,0.29,0.09,0.7,11.6,12.3,32.1,19,13,0,71,1.3,2.1,53,55,2.2,2
2204,15.6,2.04,0.33,0.11,0.8,13.6,14.4,29.6,18,10,0,74,1.1,2.15,47,41,1.7,1.6
2205,15.7,2.03,0.32,0.1,0.7,13.6,14.4,30.3,8,11,0,78,1.4,2.13,37,36,2.5,2
2206,15.9,2.01,0.28,0.08,0.5,11.8,12.3,31.9,10,9,0,79,1.6,2.09,42,53,2.3,2.1


In [245]:
# 1.2缺失值以及無效值以前後 1 小時平均值取代 (如果前 1 小時仍有空值，再取更前 1 小時)
#1.3NR 表示無降雨，以 0 取代

def preprocess_data(df_all):
    # 將 "NR" 替換為 0
    df_all.replace("NR", 0, inplace=True)

    # 將非數字的無效值轉為 NaN
    df_all = df_all.apply(pd.to_numeric, errors='coerce')

     # 前向填補和後向填補
    record_front = df_all.ffill()  # 用前一小時的值填補
    record_back = df_all.bfill()   # 用後一小時的值填補
    
    # 取填補後的平均值
    df_all_filled = (record_front + record_back) / 2

    # 將剩餘的 NaN 替換為 0
    df_all_filled.fillna(0, inplace=True)

    return df_all_filled

df_all = preprocess_data(df_all)
df_all

,AMB_TEMP,CH4,CO,NMHC,NO,NO2,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,THC,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR
0,28.3,2.04,0.34,0.17,0.9,18.8,19.8,16.0,28.0,28.0,0.0,71.0,1.8,2.21,62.0,33.0,0.8,0.7
1,28.3,2.02,0.30,0.13,0.2,11.9,12.2,21.5,24.0,22.0,0.0,66.0,1.0,2.15,121.0,183.0,1.2,0.6
2,27.8,2.12,0.30,0.12,0.5,15.1,15.6,16.9,29.0,26.0,0.0,76.0,1.0,2.24,164.0,160.0,1.1,0.6
3,27.8,2.18,0.29,0.14,0.4,12.8,13.2,16.4,32.0,24.0,0.0,79.0,1.3,2.32,156.0,151.0,0.6,0.4
4,27.6,2.19,0.30,0.17,0.2,14.9,15.1,12.6,31.0,28.0,0.0,81.0,1.5,2.36,110.0,90.0,0.9,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203,15.8,2.01,0.29,0.09,0.7,11.6,12.3,32.1,19.0,13.0,0.0,71.0,1.3,2.10,53.0,55.0,2.2,2.0
2204,15.6,2.04,0.33,0.11,0.8,13.6,14.4,29.6,18.0,10.0,0.0,74.0,1.1,2.15,47.0,41.0,1.7,1.6
2205,15.7,2.03,0.32,0.10,0.7,13.6,14.4,30.3,8.0,11.0,0.0,78.0,1.4,2.13,37.0,36.0,2.5,2.0
2206,15.9,2.01,0.28,0.08,0.5,11.8,12.3,31.9,10.0,9.0,0.0,79.0,1.6,2.09,42.0,53.0,2.3,2.1


In [247]:
#1.4 將資料切割成訓練集 (10.11月) 以及測試集 (12月)

df_train = df_all[:1464]
df_test = df_all[1464:]

In [249]:
#1.5 製作時序資料: 將資料形式轉換為行 (row) 代表18種屬性，欄 (column) 代表逐時數據資料
#hint: 將訓練集每 18 行合併，轉換成維度為 (18,61*24) 的 DataFrame (每個屬性都有 61 天 * 24 小時共1464 筆)

df_train = df_train.T
df_test = df_test.T
df_train


,0,1,2,3,4,5,6,7,8,9,...,1454,1455,1456,1457,1458,1459,1460,1461,1462,1463
AMB_TEMP,28.30,28.30,27.80,27.80,27.60,27.60,27.70,28.40,30.00,30.90,...,23.60,23.20,21.70,20.80,20.50,20.30,19.90,19.40,19.00,18.40
CH4,2.04,2.02,2.12,2.18,2.19,2.24,2.21,2.17,2.13,2.07,...,1.98,1.98,2.01,2.08,2.09,2.10,2.10,2.09,2.07,2.05
CO,0.34,0.30,0.30,0.29,0.30,0.33,0.44,0.62,0.60,0.45,...,0.21,0.22,0.31,0.44,0.46,0.46,0.45,0.42,0.37,0.33
NMHC,0.17,0.13,0.12,0.14,0.17,0.16,0.18,0.23,0.20,0.13,...,0.08,0.10,0.11,0.13,0.10,0.11,0.10,0.09,0.08,0.07
NO,0.90,0.20,0.50,0.40,0.20,0.60,2.20,3.60,2.70,1.30,...,1.80,1.60,1.50,1.10,0.80,1.10,1.00,1.10,0.90,0.80
NO2,18.80,11.90,15.10,12.80,14.90,17.50,20.00,22.10,18.10,13.20,...,7.60,9.30,12.80,13.40,12.70,12.50,10.50,8.80,8.20,7.10
NOx,19.80,12.20,15.60,13.20,15.10,18.20,22.30,25.70,20.80,14.60,...,9.50,10.90,14.40,14.50,13.60,13.70,11.50,10.00,9.20,8.00
O3,16.00,21.50,16.90,16.40,12.60,11.30,13.00,15.20,32.70,52.60,...,30.10,29.10,30.40,37.40,36.30,34.90,33.50,33.80,32.60,34.70
PM10,28.00,24.00,29.00,32.00,31.00,32.00,46.00,48.00,58.00,51.00,...,21.00,20.00,33.00,63.00,56.00,58.00,46.00,52.00,54.00,55.00
PM2.5,28.00,22.00,26.00,24.00,28.00,20.00,31.00,37.00,38.00,28.00,...,9.00,11.00,17.00,34.00,34.00,36.00,34.00,32.00,32.00,25.00


In [251]:
#2時間序列
#2.1預測目標
#cut_x 用來生成模型的輸入資料
def cut_x(dataset, interval):
    # 將 DataFrame 轉換為 NumPy 陣列，以提高切割速度
    data_array = dataset.values
    num_rows, num_cols = data_array.shape

    # 計算可以創建的切割段數
    num_segments = num_cols - interval  # 這裡不再 +1

    # 使用列表推導式生成切割的數據段
    dataset_cut = [data_array[:, i:i + interval].flatten() for i in range(num_segments)]
    
    return pd.DataFrame(dataset_cut)

def cut_y(dataset, interval, num):
    data_array = dataset.values
    num_rows, num_cols = data_array.shape

    # 計算可以為 y 創建的切割段數
    num_segments = num_cols - (interval + num - 1)

    # 使用列表推導式生成目標值
    dataset_cut = [data_array[:, i + (interval + num - 1)] for i in range(num_segments)]

    return pd.DataFrame(dataset_cut)

x_train_all = cut_x(df_train, 6)
x_train_pm25 = cut_x(df_train[df_train.index == 'PM2.5'], 6)
y_train_one = cut_y(df_train[df_train.index == 'PM2.5'], 6, 1)
y_train_six = cut_y(df_train[df_train.index == 'PM2.5'], 6, 6)

x_test_all = cut_x(df_test, 6)
x_test_pm25 = cut_x(df_test[df_test.index == 'PM2.5'], 6)
y_test_one = cut_y(df_test[df_test.index == 'PM2.5'], 6, 1)
y_test_six = cut_y(df_test[df_test.index == 'PM2.5'], 6, 6)

In [253]:
print("x_train_all:",len(x_train_all)) 
print("x_train_pm25:",len(x_train_pm25))  
print("y_train_one:", len(y_train_one)) 
print("y_train_six:",len(y_train_six)) 

x_train_all: 1458
x_train_pm25: 1458
y_train_one: 1458
y_train_six: 1453


In [256]:
# Linear Regression建模
#2.2 X請分別取
#2.3 使用兩種模型 Linear Regression 和 XGBoost 建模
#2.4用測試集資料計算 MAE (會有 8 個結果， 2 種 X 資料 * 2 種 Y 資料 * 2 種模型)

In [258]:
lr = LinearRegression(fit_intercept = False)
xgb = XGBRegressor(gamma=0.1) 

In [260]:
# LinearRegression所有屬性預測未來第一個小時
#訓練模型
lr.fit(x_train_all, y_train_one)
#進行預測
pred = lr.predict(x_test_all)
# 計算 MAE
print("LinearRegression所有屬性預測未來第一個小時:",mean_absolute_error(y_test_one, pred))


LinearRegression所有屬性預測未來第一個小時: 2.654829756888973


In [262]:
# LinearRegression只有PM2.5預測未來第一個小時
#訓練模型
lr.fit(x_train_pm25, y_train_one)
#進行預測
pred = lr.predict(x_test_pm25)
# 計算 MAE
print("LinearRegression只有PM2.5預測未來第一個小時:",mean_absolute_error(y_test_one, pred))


LinearRegression只有PM2.5預測未來第一個小時: 2.7082946116594364


In [264]:
# LinearRegression所有屬性預測未來第六個小時
#訓練模型
lr.fit(x_train_all[:-5], y_train_six)
#進行預測
pred = lr.predict(x_test_all[:-5])
# 計算 MAE
print("LinearRegression所有屬性預測未來第六個小時:",mean_absolute_error(y_test_six, pred))


LinearRegression所有屬性預測未來第六個小時: 4.30146469781756


In [266]:
# LinearRegression只有PM2.5預測未來第六個小時
#訓練模型
lr.fit(x_train_pm25[:-5], y_train_six)
#進行預測
pred = lr.predict(x_test_pm25[:-5])
# 計算 MAE
print("LinearRegression只有PM2.5預測未來第六個小時:",mean_absolute_error(y_test_six, pred))


LinearRegression只有PM2.5預測未來第六個小時: 4.706702659177684


In [268]:
# XGBRegressor所有屬性預測未來第一個小時
#訓練模型
xgb.fit(x_train_all, y_train_one)
#進行預測
pred = xgb.predict(x_test_all)
# 計算 MAE
print("XGBRegressor所有屬性預測未來第一個小時:",mean_absolute_error(y_test_one, pred))


XGBRegressor所有屬性預測未來第一個小時: 3.0852030428444466


In [270]:
# XGBRegressor只有PM2.5預測未來第一個小時
#訓練模型
xgb.fit(x_train_pm25, y_train_one)
#進行預測
pred = xgb.predict(x_test_pm25)
# 計算 MAE
print("XGBRegressor只有PM2.5預測未來第一個小時:",mean_absolute_error(y_test_one, pred))


XGBRegressor只有PM2.5預測未來第一個小時: 3.126670423723495


In [272]:
# XGBRegressor所有屬性預測未來第六個小時
#訓練模型
xgb.fit(x_train_all[:-5], y_train_six)
#進行預測
pred = xgb.predict(x_test_all[:-5])
# 計算 MAE
print("XGBRegressor所有屬性預測未來第六個小時:",mean_absolute_error(y_test_six, pred))


XGBRegressor所有屬性預測未來第六個小時: 4.881513605338977


In [274]:
# XGBRegressor只有PM2.5預測未來第六個小時
#訓練模型
xgb.fit(x_train_pm25[:-5], y_train_six)
#進行預測
pred = xgb.predict(x_test_pm25[:-5])
# 計算 MAE
print("XGBRegressor只有PM2.5預測未來第六個小時:",mean_absolute_error(y_test_six, pred))


XGBRegressor只有PM2.5預測未來第六個小時: 4.97954807600253
